In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [2]:
data = rf.earth_occ()
earth_ang = data[0]
angle_d = data[1]
area_frac = data[2]
free_area = data[3]
occ_area = data[4]

print len(earth_ang)
print len(angle_d)
print len(area_frac)
print len(free_area)
print len(occ_area)

361
361
361
361
361


In [3]:
print len(earth_ang[np.where(earth_ang > 90.)])

180


In [4]:
### to be consistent with the previous earth area ###
total_earth_area = occ_area[0][-1]

3.87245942607


In [9]:
total_angle_d = np.arange(math.pi/36000., math.pi+math.pi/36000., math.pi/36000.)

total_free_area = 2*math.pi*(1 - np.cos(total_angle_d))

total_occ_area = []
for i in range(0, len(earth_ang)):
    occ_area_addition = total_earth_area - np.flipud(occ_area[-(i+1)])
    occ_area_part = np.append(occ_area[i], occ_area_addition)
    
    total_occ_area.append(occ_area_part)
    
total_occ_area = np.array(total_occ_area)

total_area_frac = np.divide(total_occ_area, total_free_area)

In [20]:
number = len(earth_ang)
    
user = getpass.getuser()
fits_path = '/home/' + user + '/Work/earth_occultation/'
fitsname = 'earth_occ_calc_total_kt.fits'
fitsfilepath = os.path.join(fits_path, fitsname)

prihdu = fits.PrimaryHDU()
hdulist = [prihdu]

for i in range(0, number):

    angle_d = total_angle_d*180./math.pi
    area_frac = total_area_frac[i]
    free_area = total_free_area
    occ_area = total_occ_area[i]
    
    col1 = fits.Column(name = 'angle_d', format = 'E', array = angle_d, unit = 'deg')
    col2 = fits.Column(name = 'area_frac', format = 'E', array = area_frac)
    col3 = fits.Column(name = 'free_area', format = 'E', array = free_area, unit = 'rad')
    col4 = fits.Column(name = 'occ_area', format = 'E', array = occ_area, unit = 'rad')
    cols = fits.ColDefs([col1, col2, col3, col4])
    hdulist.append(fits.TableHDU.from_columns(cols, name = str(earth_ang[i])))
 
thdulist = fits.HDUList(hdulist)
thdulist.writeto(fitsfilepath)